# Machine Learning for Energy Consumption Reduction in Hybrid Energy Storage Electric Vehicles

This notebook demonstrates a machine learning approach to predict and optimize energy consumption in hybrid energy storage electric vehicles (HESS EVs).

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

## Data Generation and Preprocessing

In [ ]:
def generate_synthetic_data(num_samples=1000):
    np.random.seed(42)
    speed = np.random.uniform(0, 120, num_samples)
    acceleration = np.random.uniform(-5, 5, num_samples)
    load = np.random.uniform(0, 1000, num_samples)
    battery_soc = np.random.uniform(20, 100, num_samples)
    supercap_soc = np.random.uniform(0, 100, num_samples)
    
    energy_consumption = (0.1 * speed + 0.05 * acceleration + 0.02 * load +
                          0.01 * (100 - battery_soc) + 0.005 * (100 - supercap_soc) +
                          np.random.normal(0, 0.5, num_samples))
    
    data = pd.DataFrame({
        'speed': speed,
        'acceleration': acceleration,
        'load': load,
        'battery_soc': battery_soc,
        'supercap_soc': supercap_soc,
        'energy_consumption': energy_consumption
    })
    return data

# Generate data
data = generate_synthetic_data()
print(data.head())
print(data.describe())

In [ ]:
# Preprocess data
X = data.drop('energy_consumption', axis=1)
y = data['energy_consumption']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Exploratory Data Analysis

In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

## Model Training

In [ ]:
# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

print("Model trained successfully!")

## Model Evaluation

In [ ]:
# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Energy Consumption (kWh)')
plt.ylabel('Predicted Energy Consumption (kWh)')
plt.title('Actual vs Predicted Energy Consumption')
plt.grid(True)
plt.show()

In [ ]:
# Feature importance
feature_names = X.columns.tolist()
importances = model.feature_importances_
indices = importances.argsort()[::-1]

plt.figure(figsize=(10, 6))
plt.title('Feature Importances')
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
plt.tight_layout()
plt.show()

## Save Model

In [ ]:
# Save the model
joblib.dump(model, 'random_forest_model.pkl')
print("Model saved as 'random_forest_model.pkl'")